In [6]:
import pandas as pd
from pathlib import Path

# -------- Paths --------
OWN_FP   = Path(r"C:\Repositories\white-bowblis-nhmc\data\interim\ccn_chow_lite.csv")
MCR_FP   = Path(r"C:\Repositories\white-bowblis-nhmc\data\interim\mcr_chow_provider_events_all.csv")
HOSP_FP  = Path(r"C:\Users\Owner\OneDrive\NursingHomeData\provider-info-files\provider_resides_in_hospital_by_ccn.csv")
OUT_XLSX = Path(r"C:\Repositories\white-bowblis-nhmc\data\interim\chow_agreement_tables.xlsx")

# -------- Load --------
own  = pd.read_csv(OWN_FP, dtype={"cms_certification_number":"string"})
mcr  = pd.read_csv(MCR_FP, dtype={"cms_certification_number":"string"})
hosp = pd.read_csv(HOSP_FP, dtype={"cms_certification_number":"string"})

# Normalize hospital flag
def parse_bool(x):
    s = str(x).strip().lower()
    if s in {"1","true","t","y","yes"}:  return True
    if s in {"0","false","f","n","no"}:  return False
    return pd.NA

hosp["provider_resides_in_hospital"] = hosp["provider_resides_in_hospital"].map(parse_bool)
hosp = hosp[["cms_certification_number","provider_resides_in_hospital"]]

# Filter out hospital-resident providers
own0, mcr0 = len(own), len(mcr)
own = own.merge(hosp, on="cms_certification_number", how="left")
mcr = mcr.merge(hosp, on="cms_certification_number", how="left")
own = own[own["provider_resides_in_hospital"] != True].copy()
mcr = mcr[mcr["provider_resides_in_hospital"] != True].copy()
print(f"[hospital filter] MCR {mcr0:,} -> {len(mcr):,} | OWN {own0:,} -> {len(own):,}")

# Sanity: required columns
if "num_chows" not in own.columns:
    raise KeyError("Ownership file missing 'num_chows'.")
if "n_chow" not in mcr.columns:
    raise KeyError("MCR file missing 'n_chow'.")

# -------- Overlap on CCN --------
merged = pd.merge(
    own[["cms_certification_number","num_chows"]],
    mcr[["cms_certification_number","n_chow"]],
    on="cms_certification_number", how="inner"
)
print(f"[MERGED overlap] = {len(merged):,}")

# Collapse to 0 / 1 / 2+
def to_cat(n):
    try:
        n = int(n)
    except Exception:
        return "0"
    if n <= 0: return "0"
    if n == 1: return "1"
    return "2+"

merged["own_cat"] = merged["num_chows"].map(to_cat)
merged["mcr_cat"] = merged["n_chow"].map(to_cat)

# -------- Crosstab (0/1/2+) --------
ctab = pd.crosstab(merged["own_cat"], merged["mcr_cat"]).reindex(
    index=["0","1","2+"], columns=["0","1","2+"], fill_value=0
).reset_index().rename(columns={"own_cat":"Ownership n_chow"})
ctab["Total"] = ctab[["0","1","2+"]].sum(axis=1)

# -------- OWN-based buckets --------
own_base = merged.copy()
own_match_00 = ((own_base["own_cat"]=="0")  & (own_base["mcr_cat"]=="0")).sum()
own_match_11 = ((own_base["own_cat"]=="1")  & (own_base["mcr_cat"]=="1")).sum()
own_match_2p = ((own_base["own_cat"]=="2+") & (own_base["mcr_cat"]=="2+")).sum()
own_total    = len(own_base)
own_disc     = own_total - (own_match_00 + own_match_11 + own_match_2p)

own_bucket = pd.DataFrame({
    "Bucket": [
        "OWN=0 & MCR=0",
        "OWN=1 & MCR=1",
        "OWN=2+ & MCR=2+",
        "Discrepancies (OWN base)",
        "TOTAL (OWN base overlap)"
    ],
    "Count": [int(own_match_00), int(own_match_11), int(own_match_2p), int(own_disc), int(own_total)]
})
own_bucket["Share_of_Overlap_%"] = (
    own_bucket["Count"] / own_total * 100
).round(2)

# -------- MCR-based buckets (same overlap; just perspective check) --------
mcr_base = merged.copy()
mcr_match_00 = ((mcr_base["mcr_cat"]=="0")  & (mcr_base["own_cat"]=="0")).sum()
mcr_match_11 = ((mcr_base["mcr_cat"]=="1")  & (mcr_base["own_cat"]=="1")).sum()
mcr_match_2p = ((mcr_base["mcr_cat"]=="2+") & (mcr_base["own_cat"]=="2+")).sum()
mcr_total    = len(mcr_base)
mcr_disc     = mcr_total - (mcr_match_00 + mcr_match_11 + mcr_match_2p)

mcr_bucket = pd.DataFrame({
    "Bucket": [
        "MCR=0 & OWN=0",
        "MCR=1 & OWN=1",
        "MCR=2+ & OWN=2+",
        "Discrepancies (MCR base)",
        "TOTAL (MCR base overlap)"
    ],
    "Count": [int(mcr_match_00), int(mcr_match_11), int(mcr_match_2p), int(mcr_disc), int(mcr_total)]
})
mcr_bucket["Share_of_Overlap_%"] = (
    mcr_bucket["Count"] / mcr_total * 100
).round(2)

# -------- Discrepancies table --------
discrepancies = merged.loc[merged["own_cat"] != merged["mcr_cat"], [
    "cms_certification_number","num_chows","n_chow","own_cat","mcr_cat"
]].sort_values("cms_certification_number").reset_index(drop=True)

# -------- Overview --------
overview = pd.DataFrame({
    "Metric": [
        "MCR providers (after hospital filter)",
        "Ownership providers (after hospital filter)",
        "Overlap (inner join on CCN)"
    ],
    "Count": [len(mcr), len(own), len(merged)]
})

# -------- Print to console (so you see the buckets immediately) --------
print("\n=== OWN-based buckets (overlap base) ===")
print(own_bucket.to_string(index=False))
print("\n=== MCR-based buckets (overlap base) ===")
print(mcr_bucket.to_string(index=False))
print("\n=== Crosstab 0/1/2+ (Ownership rows × MCR cols) ===")
print(ctab.to_string(index=False))
print(f"\n[discrepancies] rows = {len(discrepancies):,}")

# -------- Excel export (with simple autofit) --------
def compute_col_widths(df, extra=2, min_w=8, max_w=60):
    widths = []
    for col in df.columns:
        series = df[col].astype(str)
        max_len = max([len(col)] + series.map(len).tolist()) + extra
        widths.append(max(min_w, min(max_w, max_len)))
    return widths

try:
    import xlsxwriter  # if present we'll use it
    engine = "xlsxwriter"
except Exception:
    engine = "openpyxl"

with pd.ExcelWriter(OUT_XLSX, engine=engine) as xw:
    overview.to_excel(xw, sheet_name="Overview", index=False)
    own_bucket.to_excel(xw, sheet_name="Buckets_OWN_base", index=False)
    mcr_bucket.to_excel(xw, sheet_name="Buckets_MCR_base", index=False)
    ctab.to_excel(xw, sheet_name="Crosstab_0-1-2plus", index=False)
    discrepancies.to_excel(xw, sheet_name="Discrepancies", index=False)

    if engine == "xlsxwriter":
        for name, df in {
            "Overview": overview,
            "Buckets_OWN_base": own_bucket,
            "Buckets_MCR_base": mcr_bucket,
            "Crosstab_0-1-2plus": ctab,
            "Discrepancies": discrepancies
        }.items():
            ws = xw.sheets[name]
            for i, w in enumerate(compute_col_widths(df)):
                ws.set_column(i, i, w)
    else:
        from openpyxl.utils import get_column_letter
        wb = xw.book
        for name, df in {
            "Overview": overview,
            "Buckets_OWN_base": own_bucket,
            "Buckets_MCR_base": mcr_bucket,
            "Crosstab_0-1-2plus": ctab,
            "Discrepancies": discrepancies
        }.items():
            ws = wb[name]
            for i, w in enumerate(compute_col_widths(df), start=1):
                ws.column_dimensions[get_column_letter(i)].width = w

print(f"\n[saved] Excel -> {OUT_XLSX}")

[hospital filter] MCR 15,124 -> 14,864 | OWN 13,419 -> 13,419
[MERGED overlap] = 13,348

=== OWN-based buckets (overlap base) ===
                  Bucket  Count  Share_of_Overlap_%
           OWN=0 & MCR=0   5945               44.54
           OWN=1 & MCR=1   1659               12.43
         OWN=2+ & MCR=2+    546                4.09
Discrepancies (OWN base)   5198               38.94
TOTAL (OWN base overlap)  13348              100.00

=== MCR-based buckets (overlap base) ===
                  Bucket  Count  Share_of_Overlap_%
           MCR=0 & OWN=0   5945               44.54
           MCR=1 & OWN=1   1659               12.43
         MCR=2+ & OWN=2+    546                4.09
Discrepancies (MCR base)   5198               38.94
TOTAL (MCR base overlap)  13348              100.00

=== Crosstab 0/1/2+ (Ownership rows × MCR cols) ===
Ownership n_chow    0    1  2+  Total
               0 5945  642  50   6637
               1 1843 1659 179   3681
              2+ 1147 1337 546   3030

In [7]:
# Subset just the discrepancies
discrepancies = merged[
    ~(
        ((merged["own_cat"] == "0") & (merged["mcr_cat"] == "0")) |
        ((merged["own_cat"] == "1") & (merged["mcr_cat"] == "1")) |
        ((merged["own_cat"] == "2+") & (merged["mcr_cat"] == "2+"))
    )
].copy()

print(f"[discrepancies] rows = {len(discrepancies):,}")

# Crosstab of where discrepancies occur
disc_xtab = pd.crosstab(discrepancies["own_cat"], discrepancies["mcr_cat"],
                        rownames=["Ownership n_chow"], colnames=["MCR n_chow"],
                        margins=True)

print("\n=== Discrepancy Crosstab (Ownership × MCR) ===")
print(disc_xtab)

# Optional: preview some example rows
print("\n=== Example discrepancy rows ===")
print(discrepancies.head(10))

[discrepancies] rows = 5,198

=== Discrepancy Crosstab (Ownership × MCR) ===
MCR n_chow           0     1   2+   All
Ownership n_chow                       
0                    0   642   50   692
1                 1843     0  179  2022
2+                1147  1337    0  2484
All               2990  1979  229  5198

=== Example discrepancy rows ===
   cms_certification_number  num_chows  n_chow own_cat mcr_cat
4                    015019          2       0      2+       0
8                    015028          3       0      2+       0
10                   015032          2       1      2+       1
14                   015040          2       1      2+       1
19                   015048          1       0       1       0
20                   015049          0       1       0       1
21                   015050          2       1      2+       1
23                   015063          2       1      2+       1
26                   015067          2       0      2+       0
29                 